In [4]:
import torch
from captcha_dataset import CaptchaData
from model import VGG

## 1. Load pytorch model: network + trained parameters

In [63]:
# 1.1 load vgg-like net
image_size = (3, 60, 120)
num_classes = 26*4
conv_pattern = ((2, 64), (2, 128), (3, 256), (3, 512), (3, 512))
hidden_pattern = (2, 512)
    
net = VGG(image_size, num_classes, conv_pattern, hidden_pattern, dropout=0.5)

In [64]:
# 1.2 load trained parameters
checkpoint = torch.load("./model.pt")  
net.load_state_dict(checkpoint)
net.eval() # validation mode

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU(inplace=True)
    (6): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (7): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (9): ReLU(inplace=True)
    (10): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (12): ReLU(inplace=True)
    (13): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (14): Conv2d(128, 256

## 2. Test pytorch model

In [65]:
# 2.1 load test data
test_dataset = CaptchaData('../samples/qq', train=False)

In [66]:
# 2.2 test pytorch model
image, label = test_dataset[0]
y_torch = net(image.unsqueeze(0))

print(y_torch)

tensor([[-25.2161, -20.8584, -22.0234, -22.0476,  10.0669, -17.6713, -25.2594,
         -27.7459, -30.1685, -28.7032, -28.6326, -21.3770, -29.2948, -26.2700,
         -21.7358, -27.6670, -26.8095, -26.8388, -22.9397, -21.0337, -21.6800,
         -22.3271, -29.8663, -24.5231, -19.7723, -28.7700, -23.4356, -30.3184,
         -24.8494, -23.7260, -24.5259, -20.2181, -20.3613, -40.3949, -36.2767,
         -27.1616, -26.2214, -31.5294, -27.9566, -30.6482, -26.4645,  10.0041,
         -20.8672, -13.8068, -28.0826, -34.1110, -27.2392, -28.0956, -24.3233,
         -32.1859, -34.9182, -28.9798, -14.9454, -23.2569, -22.4827, -15.6714,
         -22.9747, -19.2316, -15.8154, -17.0972, -30.1884, -24.2281, -28.1569,
         -17.8319, -22.0822, -23.5860, -14.1851, -20.5306,   6.1449, -15.5362,
         -21.2989, -22.6076, -14.1713, -16.8409, -22.7419, -20.7258, -25.4364,
         -23.2368, -27.5873, -27.5991, -23.6346, -26.3503, -28.6497, -25.5724,
         -25.3335, -32.4916, -58.2927, -33.6981, -29

## 3. Convert pytorch model to onnx

In [68]:
# 3.1 pytorch model to onnx model
batch_size = 1
x = torch.randn(batch_size, *image_size)
export_onnx_file = "model.onnx"
torch.onnx.export(net,
        x,
        export_onnx_file,
        opset_version=9,
        export_params=True,
        verbose=False,
        do_constant_folding=True,
        input_names=["input"],
        output_names=["output0"],
        dynamic_axes={"input":{0:"batch_size"},	"output0":{0:"batch_size"}})

In [69]:
# 3.2 check onnx model
import onnx

# Load the ONNX model
model = onnx.load("model.onnx")

# Check that the IR is well formed
onnx.checker.check_model(model)

# Print a human readable representation of the graph
print(onnx.helper.printable_graph(model.graph))

graph torch-jit-export (
  %input[FLOAT, batch_sizex3x60x120]
) initializers (
  %145[FLOAT, 64x3x3x3]
  %146[FLOAT, 64]
  %148[FLOAT, 64x64x3x3]
  %149[FLOAT, 64]
  %151[FLOAT, 128x64x3x3]
  %152[FLOAT, 128]
  %154[FLOAT, 128x128x3x3]
  %155[FLOAT, 128]
  %157[FLOAT, 256x128x3x3]
  %158[FLOAT, 256]
  %160[FLOAT, 256x256x3x3]
  %161[FLOAT, 256]
  %163[FLOAT, 256x256x3x3]
  %164[FLOAT, 256]
  %166[FLOAT, 512x256x3x3]
  %167[FLOAT, 512]
  %169[FLOAT, 512x512x3x3]
  %170[FLOAT, 512]
  %172[FLOAT, 512x512x3x3]
  %173[FLOAT, 512]
  %175[FLOAT, 512x512x3x3]
  %176[FLOAT, 512]
  %178[FLOAT, 512x512x3x3]
  %179[FLOAT, 512]
  %181[FLOAT, 512x512x3x3]
  %182[FLOAT, 512]
  %classifier.0.bias[FLOAT, 512]
  %classifier.0.weight[FLOAT, 512x1536]
  %classifier.3.bias[FLOAT, 104]
  %classifier.3.weight[FLOAT, 104x512]
) {
  %144 = Conv[dilations = [1, 1], group = 1, kernel_shape = [3, 3], pads = [1, 1, 1, 1], strides = [1, 1]](%input, %145, %146)
  %98 = Relu(%144)
  %147 = Conv[dilations = [1, 1], gr

## 4. Deploy onnx model with opencv

In [70]:
# 4.1 load onnx model 
import cv2
onnx_net = cv2.dnn.readNet('model.onnx')

In [72]:
# 4.2 test loaded onnx model
image, label = test_dataset[0]
x = image.unsqueeze(0).numpy()
onnx_net.setInput(x)
y_onnx = onnx_net.forward()

diff = (y_onnx-y_torch.detach().numpy())**2
diff = diff.sum()**0.5

print(diff)

6.522741919541013e-05
